In [28]:
%pip install numpy pandas scipy scikit-learn torch plotly kaleido umap-learn joblib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
# ============================================================
# FEATURE CONTRACT + SCALER GENERATION
# ============================================================
import os
import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler

OUTDIR = "models_disentangled"
os.makedirs(OUTDIR, exist_ok=True)

# ------------------------------------------------------------
# Load datasets
# ------------------------------------------------------------
carbon  = pd.read_excel("carbon_dataset_experimental.xlsx")
mof_exp = pd.read_excel("mof_dataset_experimental.xlsx")
mof_sim = pd.read_excel("mof_dataset_simulation.xlsx")

carbon["domain"]  = "carbon_exp"
mof_exp["domain"] = "mof_exp"
mof_sim["domain"] = "mof_sim"

df = pd.concat([carbon, mof_exp, mof_sim], ignore_index=True)

# ------------------------------------------------------------
# Define targets explicitly
# ------------------------------------------------------------
targets = [
    "Cg (F/g)",
    "Cv (F/cm^3)",
    "I_Ag",
    "I_e",
]

# Ensure targets exist
for t in targets:
    if t not in df.columns:
        df[t] = np.nan
    df[t] = pd.to_numeric(df[t], errors="coerce")

# ------------------------------------------------------------
# Define feature columns
#   Rule:
#   - numeric
#   - NOT targets
#   - NOT domain label
# ------------------------------------------------------------
exclude = set(targets + ["domain"])
feature_cols = [
    c for c in df.columns
    if c not in exclude and pd.api.types.is_numeric_dtype(df[c])
]

feature_cols = sorted(feature_cols)

print(f"🧩 Feature columns: {len(feature_cols)}")
print(f"🎯 Targets: {targets}")

# ------------------------------------------------------------
# Fit scaler (on all data, masked NaNs)
# ------------------------------------------------------------
X = df[feature_cols].apply(pd.to_numeric, errors="coerce").to_numpy(float)
X_filled = np.nan_to_num(X, nan=0.0)

scaler = StandardScaler()
scaler.fit(X_filled)

# ------------------------------------------------------------
# Save artifacts
# ------------------------------------------------------------
joblib.dump(feature_cols, os.path.join(OUTDIR, "feature_cols.pkl"))
joblib.dump(scaler,       os.path.join(OUTDIR, "feature_scaler.pkl"))
joblib.dump(targets,      os.path.join(OUTDIR, "target_cols.pkl"))

print("✅ Feature contract written to:", OUTDIR)


🧩 Feature columns: 13
🎯 Targets: ['Cg (F/g)', 'Cv (F/cm^3)', 'I_Ag', 'I_e']
✅ Feature contract written to: models_disentangled


In [30]:
# ============================================================
# MULTITASK ANN TRAINING
# ============================================================

import os, re, random
import numpy as np
import pandas as pd
import joblib
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

# ------------------ reproducibility ------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🔌 Device:", device)

# ------------------ paths (reuse existing artifacts) ------------------
OUTDIR = "models_disentangled"
MODELS_DIR = os.path.join(OUTDIR, "models")
os.makedirs(MODELS_DIR, exist_ok=True)

FEATURE_COLS_RAW = os.path.join(OUTDIR, "feature_cols.pkl")
FEATURE_SCALER   = os.path.join(OUTDIR, "feature_scaler.pkl")
TARGET_COLS      = os.path.join(OUTDIR, "target_cols.pkl")

assert os.path.exists(FEATURE_COLS_RAW)
assert os.path.exists(FEATURE_SCALER)
assert os.path.exists(TARGET_COLS)

feature_cols = joblib.load(FEATURE_COLS_RAW)
xsc          = joblib.load(FEATURE_SCALER)
targets      = joblib.load(TARGET_COLS)

# ------------------ load datasets ------------------
carbon  = pd.read_excel("carbon_dataset_experimental.xlsx")
mof_exp = pd.read_excel("mof_dataset_experimental.xlsx")
mof_sim = pd.read_excel("mof_dataset_simulation.xlsx")

carbon["domain"]  = "carbon_exp"
mof_exp["domain"] = "mof_exp"
mof_sim["domain"] = "mof_sim"

df = pd.concat([carbon, mof_exp, mof_sim], ignore_index=True)

# ensure targets exist
for t in targets:
    df[t] = pd.to_numeric(df.get(t, np.nan), errors="coerce")

# ------------------ build ANN input ------------------
X_df = df.reindex(columns=feature_cols).apply(pd.to_numeric, errors="coerce")
X_num = X_df.to_numpy(float)

X_mask = np.isfinite(X_num).astype(np.float32)
X_scaled = xsc.transform(np.nan_to_num(X_num, nan=0.0)).astype(np.float32)

Xs_ann = np.concatenate([X_scaled, X_mask], axis=1)
Y = df[targets].to_numpy(np.float32)

print("🧠 ANN input shape:", Xs_ann.shape)

# ------------------ train/val split ------------------
idx_all = np.arange(len(df))
idx_tr, idx_va = train_test_split(idx_all, test_size=0.2, random_state=SEED)

# ------------------ model ------------------
class MultiTaskANN(nn.Module):
    """
    Paper A encoder:
    - deterministic latent space
    - simple supervised heads
    - NO uncertainty heads
    - NO domain adversarial head
    """
    def __init__(self, in_dim, hidden, latent, targets):
        super().__init__()
        self.targets = list(targets)

        self.fc1 = nn.Linear(in_dim, hidden)
        self.fc2 = nn.Linear(hidden, latent)

        self.heads = nn.ModuleDict({
            t: nn.Linear(latent, 1) for t in self.targets
        })

    def encode(self, x):
        h = F.relu(self.fc1(x))
        z = F.relu(self.fc2(h))
        return z

    def forward(self, x):
        z = self.encode(x)
        return {t: self.heads[t](z) for t in self.targets}

model = MultiTaskANN(
    in_dim=Xs_ann.shape[1],
    hidden=128,
    latent=64,
    targets=targets
).to(device)

opt = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

# ------------------ training ------------------
X_t = torch.tensor(Xs_ann, dtype=torch.float32, device=device)
Y_t = torch.tensor(Y, dtype=torch.float32, device=device)

for epoch in range(40):
    model.train()
    opt.zero_grad()

    preds = model(X_t[idx_tr])
    loss = 0.0
    n = 0

    for j, t in enumerate(targets):
        y = Y_t[idx_tr, j:j+1]
        m = torch.isfinite(y)
        if m.sum() > 10:
            loss += loss_fn(preds[t][m], y[m])
            n += 1

    loss = loss / max(n, 1)
    loss.backward()
    opt.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch:02d} | loss={loss.item():.4f}")

# ------------------ save checkpoint ------------------
ckpt_path = os.path.join(MODELS_DIR, "multitask_ann_PAPER_A.pt")

torch.save(
    {
        "state_dict": model.state_dict(),
        "feature_dim": Xs_ann.shape[1],
        "hidden": 128,
        "latent": 64,
        "dropout": 0.0,
        "targets": targets,
        "domains": sorted(df["domain"].unique().tolist()),
    },
    ckpt_path
)

print("✅ Saved  ANN checkpoint:")
print("   ", ckpt_path)


🔌 Device: cpu
🧠 ANN input shape: (242, 26)
Epoch 00 | loss=19489.4355
Epoch 10 | loss=19364.4355
Epoch 20 | loss=19150.7402
Epoch 30 | loss=18696.5820
✅ Saved  ANN checkpoint:
    models_disentangled\models\multitask_ann_PAPER_A.pt


In [ ]:
# ======================================================================
# paperA_latent_transfer
# ======================================================================

from __future__ import annotations

import os
import re
import glob
import warnings
import importlib.util
from pathlib import Path

import numpy as np
import pandas as pd
import joblib

warnings.filterwarnings("ignore")

# -------------------- torch --------------------
import torch
import torch.nn as nn
import torch.nn.functional as F

# -------------------- sklearn / umap --------------------
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances
import umap.umap_ as umap

# -------------------- plotly --------------------
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.templates.default = "plotly_white"
pio.renderers.default = "vscode" if os.environ.get("VSCODE_PID") else "browser"

# =========================================================
# REPRODUCIBILITY
# =========================================================
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

# =========================================================
# EXPORT SETTINGS
# =========================================================
PRINT_INCHES_W = 7.0
TARGET_DPI = 300
EXPORT_WIDTH_PX = int(round(PRINT_INCHES_W * TARGET_DPI))
EXPORT_HEIGHT_PX = int(round(EXPORT_WIDTH_PX * 0.75))
EXPORT_SCALE = 1

# PNG export requires kaleido
if importlib.util.find_spec("kaleido") is None:
    raise RuntimeError("❌ Missing dependency: kaleido.")

# =========================================================
# GLOBAL STYLE
# =========================================================
FONT_BODY = "Arial"
FONT_TICK_BOLD = "Arial Black" 
TITLE_SIZE = 45
AXIS_TITLE_SIZE = 40
TICK_SIZE = 40
TICK_SIZE_3D = 26

LEGEND_SIZE = 40
LEGEND_TITLE_SIZE = 45

AXIS_LINE_WIDTH = 7
MARKER_SIZE = 10
MARKER_LINE_WIDTH = 1.5
LINE_WIDTH = 7

def bold(s: str) -> str:
    return f"<b>{s}</b>"


# =========================================================
# HELPERS
# =========================================================
def sanitize(s: str) -> str:
    return re.sub(r"[^A-Za-z0-9]+", "_", str(s)).strip("_")


def ensure_exists(df: pd.DataFrame, col: str, fill=np.nan) -> None:
    if col not in df.columns:
        df[col] = fill


def ensure_col(df: pd.DataFrame, src_name: str, dst_name: str) -> None:
    """Map a column by normalized name match (handles μ/µ/u)."""
    if dst_name in df.columns:
        return
    src_norm = src_name.lower().replace("μ", "u").replace("µ", "u").strip()
    for c in df.columns:
        c_norm = c.lower().replace("μ", "u").replace("µ", "u").strip()
        if c_norm == src_norm:
            df[dst_name] = df[c]
            return


def corr_1d(a, b) -> float:
    a = np.asarray(a, float)
    b = np.asarray(b, float)
    m = np.isfinite(a) & np.isfinite(b)
    if m.sum() < 10:
        return np.nan
    return float(np.corrcoef(a[m], b[m])[0, 1])


def export_plotly(fig: go.Figure, outdir: Path, stem: str, show: bool = True) -> None:
    outdir.mkdir(parents=True, exist_ok=True)
    html = outdir / f"{sanitize(stem)}.html"
    png = outdir / f"{sanitize(stem)}.png"

    for p in (html, png):
        if p.exists():
            try:
                p.unlink()
            except PermissionError as e:
                raise RuntimeError(f"❌ Cannot overwrite file (is it open?): {p}") from e

    fig.update_layout(width=EXPORT_WIDTH_PX, height=EXPORT_HEIGHT_PX)

    if show:
        fig.show()

    fig.write_html(str(html))
    fig.write_image(str(png), width=EXPORT_WIDTH_PX, height=EXPORT_HEIGHT_PX, scale=EXPORT_SCALE)
    print("💾 Saved:", png)


def heatmap_annotations(
    fig: go.Figure,
    Z: np.ndarray,
    xlabels: list[str],
    ylabels: list[str],
    fmt: str = ".2f",
    font_size: int = 14,
) -> None:
    """
    Overlay numeric annotations on a heatmap.
    """
    for i, y in enumerate(ylabels):
        for j, x in enumerate(xlabels):
            val = Z[i, j]
            if not np.isfinite(val):
                continue
            fig.add_annotation(
                x=x,
                y=y,
                text=f"{val:{fmt}}",
                showarrow=False,
                font=dict(family="Arial Black", size=font_size, color="black"),
            )


# =========================================================
# STYLING
# =========================================================
def style_2d(fig: go.Figure, title: str, x: str | None = None, y: str | None = None, showlegend: bool = True):
    fig.update_layout(
        title=dict(text=bold(title), x=0.5, font=dict(size=TITLE_SIZE)),
        font=dict(family=FONT_BODY),
        showlegend=showlegend,
        legend=dict(
            font=dict(size=LEGEND_SIZE),
            title=dict(font=dict(size=LEGEND_TITLE_SIZE)),
            itemsizing="constant",
        ),
        margin=dict(l=90, r=50, t=110, b=90),
    )
    fig.update_xaxes(
        title=dict(text=bold(x) if x else None, font=dict(size=AXIS_TITLE_SIZE)),
        tickfont=dict(family=FONT_TICK_BOLD, size=TICK_SIZE),
        showline=True,
        linewidth=AXIS_LINE_WIDTH,
        nticks=6,
    )
    fig.update_yaxes(
        title=dict(text=bold(y) if y else None, font=dict(size=AXIS_TITLE_SIZE)),
        tickfont=dict(family=FONT_TICK_BOLD, size=TICK_SIZE),
        showline=True,
        linewidth=AXIS_LINE_WIDTH,
        nticks=6,
    )
    return fig


def style_3d(fig: go.Figure, title: str):
    fig.update_layout(
        title=dict(text=bold(title), x=0.5, font=dict(size=TITLE_SIZE)),
        legend=dict(
            font=dict(size=LEGEND_SIZE),
            title=dict(font=dict(size=LEGEND_TITLE_SIZE)),
            itemsizing="constant",
        ),
        margin=dict(l=140, r=140, t=140, b=120),
        scene=dict(
            aspectmode="cube",
            camera=dict(eye=dict(x=1.6, y=1.6, z=1.3)),

            xaxis=dict(
                title=dict(
                    text=bold("PC1<br><br>"),
                    font=dict(size=AXIS_TITLE_SIZE),
                ),
                tickfont=dict(family=FONT_TICK_BOLD, size=TICK_SIZE_3D),
                nticks=5,
            ),

            yaxis=dict(
                title=dict(
                    text=bold("PC2<br><br>"),
                    font=dict(size=AXIS_TITLE_SIZE),
                ),
                tickfont=dict(family=FONT_TICK_BOLD, size=TICK_SIZE_3D),
                nticks=5,
            ),

            zaxis=dict(
                title=dict(
                    text=bold("PC3<br><br>"),
                    font=dict(size=AXIS_TITLE_SIZE),
                ),
                tickfont=dict(family=FONT_TICK_BOLD, size=TICK_SIZE_3D),
                nticks=5,
            ),
        ),
    )
    return fig


def style_heatmap(fig: go.Figure, title: str, x: str | None = None, y: str | None = None):
    fig.update_layout(
        title=dict(text=bold(title), x=0.5, font=dict(size=TITLE_SIZE)),
        legend=dict(
            font=dict(size=LEGEND_SIZE),
            title=dict(font=dict(size=LEGEND_TITLE_SIZE)),
            itemsizing="constant",
        ),
        margin=dict(l=120, r=70, t=110, b=120),
        font=dict(family=FONT_BODY),
    )
    fig.update_xaxes(
        title=dict(text=bold(x) if x else None, font=dict(size=AXIS_TITLE_SIZE)),
        tickfont=dict(family=FONT_TICK_BOLD, size=TICK_SIZE),
    )
    fig.update_yaxes(
        title=dict(text=bold(y) if y else None, font=dict(size=AXIS_TITLE_SIZE)),
        tickfont=dict(family=FONT_TICK_BOLD, size=TICK_SIZE),
    )

    fig.update_traces(colorbar=dict(tickfont=dict(family=FONT_TICK_BOLD, size=TICK_SIZE)))

    return fig


# =========================================================
# PATHS
# =========================================================
OUTDIR = Path("models_disentangled")
MODELS_DIR = OUTDIR / "models"
DIAG_DIR = OUTDIR / "diagnostics"
PAPER_OUT = DIAG_DIR / "paperA_latent_transfer"
PAPER_OUT.mkdir(parents=True, exist_ok=True)

FEATURE_COLS = OUTDIR / "feature_cols.pkl"
FEATURE_SCALER = OUTDIR / "feature_scaler.pkl"
TARGET_COLS = OUTDIR / "target_cols.pkl"

for p in (FEATURE_COLS, FEATURE_SCALER, TARGET_COLS):
    if not p.exists():
        raise FileNotFoundError(f"❌ Missing required artifact: {p}")

feature_cols = joblib.load(FEATURE_COLS)
xsc = joblib.load(FEATURE_SCALER)
targets = list(joblib.load(TARGET_COLS))

# =========================================================
# LOAD CHECKPOINT
# =========================================================
CKPT_PATH = MODELS_DIR / "multitask_ann_PAPER_A.pt"
if not CKPT_PATH.exists():
    cands = sorted(MODELS_DIR.glob("multitask_ann_*.pt"))
    if not cands:
        raise FileNotFoundError(f"❌ Missing checkpoint in {MODELS_DIR}")
    CKPT_PATH = cands[0]
    print("⚠️ Falling back to checkpoint:", CKPT_PATH.name)

ckpt = torch.load(str(CKPT_PATH), map_location="cpu")

for k in ("feature_dim", "hidden", "latent", "state_dict"):
    if k not in ckpt:
        raise KeyError(f"❌ Checkpoint missing key: {k}")


class MultiTaskANN(nn.Module):
    def __init__(self, in_dim: int, hidden: int, latent: int):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, hidden)
        self.fc2 = nn.Linear(hidden, latent)

    def encode(self, x: torch.Tensor) -> torch.Tensor:
        return F.relu(self.fc2(F.relu(self.fc1(x))))


ann = MultiTaskANN(int(ckpt["feature_dim"]), int(ckpt["hidden"]), int(ckpt["latent"]))
ann.load_state_dict(ckpt["state_dict"], strict=False)
ann.eval()

# =========================================================
# LOAD + HARMONIZE DATA
# =========================================================
carbon_path = Path("carbon_dataset_experimental.xlsx")
mof_sim_path = Path("mof_dataset_simulation.xlsx")
mof_exp_path = Path("mof_dataset_experimental.xlsx")

for p in (carbon_path, mof_sim_path, mof_exp_path):
    if not p.exists():
        raise FileNotFoundError(f"❌ Missing dataset file: {p.resolve()}")

carbon = pd.read_excel(carbon_path)
mof_sim = pd.read_excel(mof_sim_path)
mof_exp = pd.read_excel(mof_exp_path)


def _normalize(df: pd.DataFrame) -> pd.DataFrame:
    return df.rename(
        columns={
            "SSA (m^2/g)": "SSA (m2/g)",
            "PW(V)": "PW (V)",
        }
    )


carbon = _normalize(carbon)
mof_exp = _normalize(mof_exp)
mof_sim = _normalize(mof_sim)

carbon["domain"] = "carbon_exp"
mof_exp["domain"] = "mof_exp"
mof_sim["domain"] = "mof_sim"

# map common target variants
ensure_col(carbon, "I (A/g)", "I_Ag")
ensure_col(mof_exp, "I (A/g)", "I_Ag")
ensure_col(mof_sim, "I (e/μs/Da)", "I_e")
ensure_col(mof_sim, "I (e/µs/Da)", "I_e")
ensure_col(mof_sim, "I (e/us/Da)", "I_e")

df_all = pd.concat([carbon, mof_exp, mof_sim], ignore_index=True)

# ensure targets exist and numeric
for t in targets:
    ensure_exists(df_all, t, np.nan)
    df_all[t] = pd.to_numeric(df_all[t], errors="coerce")

# =========================================================
# ANN INPUT
# =========================================================
X_df = df_all.reindex(columns=feature_cols).apply(pd.to_numeric, errors="coerce")
X = X_df.to_numpy(float)

mask = np.isfinite(X).astype(np.float32)

X_filled = np.nan_to_num(X, nan=0.0)
Xs_scaled = xsc.transform(X_filled).astype(np.float32)
Xs = np.concatenate([Xs_scaled, mask], axis=1).astype(np.float32)

expected_in = int(ckpt["feature_dim"])
if Xs.shape[1] != expected_in:
    raise ValueError(
        f"❌ Feature dim mismatch: Xs has {Xs.shape[1]} cols, checkpoint expects {expected_in}. "
        f"(len(feature_cols)={len(feature_cols)}; scaled={Xs_scaled.shape[1]}, mask={mask.shape[1]})."
    )

with torch.no_grad():
    z = ann.encode(torch.tensor(Xs, dtype=torch.float32)).cpu().numpy()

df_all["latent_norm"] = np.linalg.norm(z, axis=1)

np.save(PAPER_OUT / "latents_z.npy", z)
df_all.to_csv(PAPER_OUT / "unified_dataframe.csv", index=False)

# =========================================================
# PCA + UMAP
# =========================================================
zp = PCA(n_components=3, svd_solver="randomized", random_state=SEED).fit_transform(z)

n = len(z)
if n < 3:
    raise ValueError(f"❌ Need at least 3 samples for UMAP; got n={n}")
n_neighbors = min(30, max(2, n - 1))
zu = umap.UMAP(n_neighbors=n_neighbors, min_dist=0.10, random_state=SEED).fit_transform(z)

pca_df = pd.DataFrame(zp, columns=["PC1", "PC2", "PC3"])
pca_df["domain"] = df_all["domain"].astype(str).values

umap_df = pd.DataFrame(zu, columns=["UMAP1", "UMAP2"])
umap_df["domain"] = df_all["domain"].astype(str).values

pca_df.to_csv(PAPER_OUT / "pca_3d_coords.csv", index=False)
umap_df.to_csv(PAPER_OUT / "umap_2d_coords.csv", index=False)

domain_map = {
    "carbon_exp": "Carbon (exp)",
    "mof_exp": "MOF (exp)",
    "mof_sim": "MOF (sim)",
}
pca_df["domain_label"] = pca_df["domain"].map(domain_map).fillna(pca_df["domain"])
umap_df["domain_label"] = umap_df["domain"].map(domain_map).fillna(umap_df["domain"])

# =========================================================
# FIG 1A — PCA3 domain
# =========================================================
fig = px.scatter_3d(pca_df, x="PC1", y="PC2", z="PC3", color="domain_label", opacity=0.85)
fig.update_traces(marker=dict(size=MARKER_SIZE, line=dict(width=MARKER_LINE_WIDTH, color="black")))
fig = style_3d(fig, "Latent Feasibility Space (3D PCA), by domain")
export_plotly(fig, PAPER_OUT, "Fig1A_latent_PCA3_domain")

# =========================================================
# FIG 1B — UMAP domain
# =========================================================
fig = px.scatter(umap_df, x="UMAP1", y="UMAP2", color="domain_label", opacity=0.85)
fig.update_traces(marker=dict(size=MARKER_SIZE, line=dict(width=MARKER_LINE_WIDTH, color="black")))
fig = style_2d(fig, "Latent Feasibility Space (2D UMAP), by domain", "UMAP1", "UMAP2", showlegend=True)
export_plotly(fig, PAPER_OUT, "Fig1B_latent_UMAP2_domain")

# =========================================================
# FIG 2 — UMAP colored by targets
# =========================================================
def plot_umap_target(t: str) -> None:
    y = pd.to_numeric(df_all[t], errors="coerce").to_numpy(float)
    tmp = umap_df[["UMAP1", "UMAP2", "domain_label"]].copy()
    tmp[t] = y

    fig = px.scatter(tmp, x="UMAP1", y="UMAP2", color=t, opacity=0.92, hover_data=["domain_label"])
    fig.update_traces(marker=dict(size=MARKER_SIZE, line=dict(width=MARKER_LINE_WIDTH, color="black")))
    fig = style_2d(fig, f"UMAP colored by {t}", "UMAP1", "UMAP2", showlegend=False)

    # ✅ bold colorbar ticks for scatter color scale
    fig.update_layout(coloraxis_colorbar=dict(
        tickfont=dict(family=FONT_TICK_BOLD, size=TICK_SIZE),
        title=dict(font=dict(size=AXIS_TITLE_SIZE))
    ))

    export_plotly(fig, PAPER_OUT, f"Fig2_UMAP_color_{sanitize(t)}")


for t in ["Cg (F/g)", "Cv (F/cm^3)", "I_Ag", "I_e"]:
    if t in df_all.columns:
        plot_umap_target(t)

# =========================================================
# FIG 3 — Domain divergence metrics
# =========================================================
def rbf_kernel_mmd2(X: np.ndarray, Y: np.ndarray, gamma: float | None = None) -> float:
    X = np.asarray(X, float)
    Y = np.asarray(Y, float)
    if X.shape[0] < 5 or Y.shape[0] < 5:
        return np.nan

    Z = np.vstack([X, Y])
    D = pairwise_distances(Z, Z, metric="euclidean")
    if gamma is None:
        med = np.median(D[D > 0])
        if not np.isfinite(med) or med <= 0:
            med = 1.0
        gamma = 1.0 / (med * med)

    def K(A: np.ndarray, B: np.ndarray) -> np.ndarray:
        DA = pairwise_distances(A, B, metric="euclidean")
        return np.exp(-gamma * (DA ** 2))

    Kxx = K(X, X)
    Kyy = K(Y, Y)
    Kxy = K(X, Y)

    np.fill_diagonal(Kxx, 0.0)
    np.fill_diagonal(Kyy, 0.0)

    m = X.shape[0]
    n = Y.shape[0]
    return float((Kxx.sum()/(m*(m-1)+1e-12)) + (Kyy.sum()/(n*(n-1)+1e-12)) - 2.0*Kxy.mean())


MAX_MMD_SAMPLES = 1500
_rs = np.random.RandomState(SEED)

def _subsample(A: np.ndarray) -> np.ndarray:
    if A.shape[0] > MAX_MMD_SAMPLES:
        idx = _rs.choice(A.shape[0], MAX_MMD_SAMPLES, replace=False)
        return A[idx]
    return A


domains = df_all["domain"].astype(str).values
uniq = sorted(pd.unique(domains).tolist())

centroids = {d: z[domains == d].mean(axis=0) for d in uniq}
cent_mat = np.full((len(uniq), len(uniq)), np.nan, float)
mmd_mat  = np.full((len(uniq), len(uniq)), np.nan, float)

for i, a in enumerate(uniq):
    Za = z[domains == a]
    for j, b in enumerate(uniq):
        Zb = z[domains == b]
        cent_mat[i, j] = float(np.linalg.norm(centroids[a] - centroids[b]))
        mmd_mat[i, j]  = rbf_kernel_mmd2(_subsample(Za), _subsample(Zb))

cent_df = pd.DataFrame(cent_mat, index=uniq, columns=uniq)
mmd_df  = pd.DataFrame(mmd_mat,  index=uniq, columns=uniq)

cent_df.to_csv(PAPER_OUT / "centroid_distance_matrix.csv", index=True)
mmd_df.to_csv(PAPER_OUT / "mmd_matrix.csv", index=True)

fig = go.Figure(go.Heatmap(
    z=cent_df.values,
    x=cent_df.columns,
    y=cent_df.index,
    colorscale="Viridis",
    colorbar=dict(title=bold("Centroid distance"),
                  tickfont=dict(family=FONT_TICK_BOLD, size=TICK_SIZE)),
))
fig = style_heatmap(fig, "Domain divergence (Centroid distance)", "Domain", "Domain")
heatmap_annotations(fig, cent_df.values, list(cent_df.columns), list(cent_df.index), fmt=".2f", font_size=35)
export_plotly(fig, PAPER_OUT, "Fig3A_centroid_distance_heatmap")

fig = go.Figure(go.Heatmap(
    z=mmd_df.values,
    x=mmd_df.columns,
    y=mmd_df.index,
    colorscale="Inferno",
    colorbar=dict(title=bold("MMD²"),
                  tickfont=dict(family=FONT_TICK_BOLD, size=TICK_SIZE)),
))
fig = style_heatmap(fig, "Domain divergence (MMD², RBF kernel)", "Domain", "Domain")
heatmap_annotations(fig, mmd_df.values, list(mmd_df.columns), list(mmd_df.index), fmt=".3f", font_size=35)
export_plotly(fig, PAPER_OUT, "Fig3B_mmd_heatmap")

# =========================================================
# FIG 4 — Latent–target coupling heatmaps (GLOBAL + per domain)
# =========================================================
def latent_target_corr_heatmap(Z: np.ndarray, df: pd.DataFrame, target_list: list[str], title: str, stem: str) -> None:
    Z = np.asarray(Z, float)

    rows, names = [], []
    for t in target_list:
        if t not in df.columns:
            continue
        y = pd.to_numeric(df[t], errors="coerce").to_numpy(float)
        if np.isfinite(y).sum() < 20:
            continue
        rows.append([corr_1d(Z[:, k], y) for k in range(Z.shape[1])])
        names.append(t)

    if not rows:
        print("⚠️ Skipping corr heatmap:", title)
        return

    C = pd.DataFrame(rows, index=names, columns=[f"z{k}" for k in range(Z.shape[1])])
    C.to_csv(PAPER_OUT / f"{sanitize(stem)}.csv", index=True)

    fig = go.Figure(go.Heatmap(
        z=C.values,
        x=C.columns,
        y=C.index,
        zmin=-1, zmax=1,
        colorbar=dict(title=bold("Pearson r"),
                      tickfont=dict(family=FONT_TICK_BOLD, size=TICK_SIZE)),  # ✅ bold colorbar ticks
    ))
    fig = style_heatmap(fig, title, "Latent dim", "Target")
    export_plotly(fig, PAPER_OUT, stem)


latent_target_corr_heatmap(
    z, df_all, targets,
    "Latent ↔ Target coupling (GLOBAL)",
    "Fig4A_latent_target_corr_GLOBAL"
)

for dom in uniq:
    ddf = df_all[df_all["domain"].astype(str) == dom].copy()
    if len(ddf) < 25:
        continue
    idx = ddf.index.to_numpy()
    latent_target_corr_heatmap(
        z[idx], ddf, targets,
        f"Latent ↔ Target coupling (domain={dom})",
        f"Fig4B_latent_target_corr_{sanitize(dom)}"
    )

# =========================================================
# FIG 5 — Property-sorted trajectories
# =========================================================
def plot_traj(coords2d: np.ndarray, df_subset: pd.DataFrame, target: str, title: str, stem: str):
    if target not in df_subset.columns:
        print(f"⚠️ Fig5 skip (missing target): {target}")
        return

    y = pd.to_numeric(df_subset[target], errors="coerce").to_numpy(float)
    m = np.isfinite(y)
    if m.sum() < 25:
        print(f"⚠️ Fig5 skip (insufficient labels for {target}: {m.sum()})")
        return

    coords = coords2d[m]
    vals = y[m]

    order = np.argsort(vals)
    coords = coords[order]
    vals = vals[order]

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=coords[:, 0],
        y=coords[:, 1],
        mode="lines",
        line=dict(color="rgba(0,0,0,0.25)", width=2),
        showlegend=False
    ))

    fig.add_trace(go.Scatter(
        x=coords[:, 0],
        y=coords[:, 1],
        mode="markers",
        marker=dict(
            size=MARKER_SIZE + 1,
            color=vals,
            colorscale="Viridis",
            showscale=True,
            colorbar=dict(title=bold(target),
                          tickfont=dict(family=FONT_TICK_BOLD, size=TICK_SIZE)),  # ✅ bold colorbar ticks
            line=dict(width=MARKER_LINE_WIDTH, color="black"),
        ),
        showlegend=False
    ))

    fig = style_2d(fig, title, "UMAP1", "UMAP2", showlegend=False)
    export_plotly(fig, PAPER_OUT, stem)


for t in ["Cg (F/g)", "Cv (F/cm^3)", "I_Ag", "I_e"]:
    if t in df_all.columns:
        plot_traj(
            coords2d=zu,
            df_subset=df_all.reset_index(drop=True),
            target=t,
            title=f"UMAP trajectory sorted by {t} (GLOBAL)",
            stem=f"Fig5A_trajectory_umap_GLOBAL_{sanitize(t)}"
        )

for dom in uniq:
    idx = np.where(df_all["domain"].astype(str).values == dom)[0]
    if idx.size < 40:
        continue

    ddf = df_all.iloc[idx].reset_index(drop=True)
    coords_dom = zu[idx]

    for t in ["Cg (F/g)", "Cv (F/cm^3)", "I_Ag", "I_e"]:
        if t in ddf.columns:
            plot_traj(
                coords2d=coords_dom,
                df_subset=ddf,
                target=t,
                title=f"UMAP trajectory sorted by {t} (domain={dom})",
                stem=f"Fig5B_trajectory_umap_{sanitize(dom)}_{sanitize(t)}"
            )

# =========================================================
# DONE
# =========================================================
print("\n✅ Figure suite complete.")
print("📁 Outputs in:", str(PAPER_OUT.resolve()))
print("🖼️ PNG export size:", EXPORT_WIDTH_PX, "x", EXPORT_HEIGHT_PX, "px")
print(f"ℹ️ DPI-equivalent if placed at {PRINT_INCHES_W:.1f}\" width: ~{EXPORT_WIDTH_PX / PRINT_INCHES_W:.0f} dpi")


💾 Saved: models_disentangled\diagnostics\paperA_latent_transfer\Fig1A_latent_PCA3_domain.png


💾 Saved: models_disentangled\diagnostics\paperA_latent_transfer\Fig1B_latent_UMAP2_domain.png


💾 Saved: models_disentangled\diagnostics\paperA_latent_transfer\Fig2_UMAP_color_Cg_F_g.png


💾 Saved: models_disentangled\diagnostics\paperA_latent_transfer\Fig2_UMAP_color_Cv_F_cm_3.png


💾 Saved: models_disentangled\diagnostics\paperA_latent_transfer\Fig2_UMAP_color_I_Ag.png


...

💾 Saved: models_disentangled\diagnostics\paperA_latent_transfer\Fig5B_trajectory_umap_carbon_exp_Cv_F_cm_3.png


💾 Saved: models_disentangled\diagnostics\paperA_latent_transfer\Fig5B_trajectory_umap_carbon_exp_I_Ag.png
⚠️ Fig5 skip (insufficient labels for I_e: 0)


💾 Saved: models_disentangled\diagnostics\paperA_latent_transfer\Fig5B_trajectory_umap_mof_sim_Cg_F_g.png


💾 Saved: models_disentangled\diagnostics\paperA_latent_transfer\Fig5B_trajectory_umap_mof_sim_Cv_F_cm_3.png
⚠️ Fig5 skip (insufficient labels for I_Ag: 0)


💾 Saved: models_disentangled\diagnostics\paperA_latent_transfer\Fig5B_trajectory_umap_mof_sim_I_e.png

✅ Figure suite complete.
📁 Outputs in: C:\Users\hrnbe\Greenbootcamps\Projects\Carbon&MOF\Paper 3\Paper A\models_disentangled\diagnostics\paperA_latent_transfer
🖼️ PNG export size: 2100 x 1575 px
ℹ️ DPI-equivalent if placed at 7.0" width: ~300 dpi


In [ ]:
# =============================================================================
# Domain Separability + Latent Dimension Importance
# =============================================================================

from __future__ import annotations

import os
import re
import warnings
import importlib.util
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings("ignore")


# =========================================================
# RENDERER
# =========================================================
if os.environ.get("VSCODE_PID"):
    pio.renderers.default = "vscode"
else:
    # prefer notebook if available, otherwise browser
    pio.renderers.default = "notebook_connected" if "ipykernel" in globals() else "browser"

pio.templates.default = "plotly_white"


# =========================================================
# EXPORT DEPENDENCY CHECK
# =========================================================
if importlib.util.find_spec("kaleido") is None:
    raise RuntimeError("❌ Missing dependency: kaleido. Install with: pip install kaleido")


# =========================================================
# PATHS
# =========================================================
PAPER_OUT = Path("models_disentangled/diagnostics/paperA_latent_transfer")

LATENT_Z = PAPER_OUT / "latents_z.npy"
DF_ALL = PAPER_OUT / "unified_dataframe.csv"
UMAP_COORDS = PAPER_OUT / "umap_2d_coords.csv"

for p in (LATENT_Z, DF_ALL, UMAP_COORDS):
    if not p.exists():
        raise FileNotFoundError(f"❌ Missing required file: {p.resolve()}")


# =========================================================
# LOAD DATA
# =========================================================
z = np.load(LATENT_Z)
df_all = pd.read_csv(DF_ALL)
umap_df = pd.read_csv(UMAP_COORDS)

if len(z) != len(df_all) or len(df_all) != len(umap_df):
    raise ValueError(
        f"❌ Length mismatch: latents={len(z)}, df_all={len(df_all)}, umap_df={len(umap_df)}"
    )


# =========================================================
# STYLE CONSTANTS
# =========================================================
FONT_FAMILY_BOLD = "Arial Black"
FONT_FAMILY_BODY = "Arial"

TITLE_SIZE = 45
AXIS_TITLE_SIZE = 40
TICK_SIZE = 25

LEGEND_SIZE = 40
LEGEND_TITLE_SIZE = 45

AXIS_LINE_WIDTH = 7
MARKER_SIZE = 10
MARKER_LINE_WIDTH = 1.5
LINE_WIDTH = 7

BAR_EDGE_WIDTH = 2
GRID_WIDTH = 1.5

COLORBAR_TITLE_SIZE = 38
COLORBAR_TICK_SIZE = 28
COLORBAR_THICKNESS = 40


# =========================================================
# EXPORT SETTINGS
# =========================================================
PRINT_INCHES_W = 10.0
TARGET_DPI = 180

EXPORT_WIDTH_PX = int(TARGET_DPI * PRINT_INCHES_W)
EXPORT_HEIGHT_PX = int(EXPORT_WIDTH_PX * 0.75)
EXPORT_SCALE = 1


# =========================================================
# HELPERS
# =========================================================
def sanitize(s: str) -> str:
    return re.sub(r"[^A-Za-z0-9]+", "_", str(s)).strip("_")


def _bold(s: str) -> str:
    return f"<b>{s}</b>"


def safe_unlink(path: Path) -> None:
    """Delete a file if it exists; raise a clear error if it cannot be overwritten."""
    if path.exists():
        try:
            path.unlink()
        except PermissionError as e:
            raise RuntimeError(f"❌ Cannot overwrite file (is it open?): {path}") from e


def export_plotly(fig: go.Figure, outdir: Path, stem: str, show: bool = True):
    outdir.mkdir(parents=True, exist_ok=True)
    png = outdir / f"{sanitize(stem)}.png"
    html = outdir / f"{sanitize(stem)}.html"

    safe_unlink(png)
    safe_unlink(html)

    fig.update_layout(
        width=EXPORT_WIDTH_PX,
        height=EXPORT_HEIGHT_PX,
        xaxis=dict(automargin=True),
        yaxis=dict(automargin=True),
    )

    if show:
        fig.show()

    fig.write_html(str(html))
    fig.write_image(str(png), scale=EXPORT_SCALE)
    print("💾 Saved:", png.name)


def style_sup_2d(fig, title, x_title=None, y_title=None):
    fig.update_layout(
        title=dict(
            text=_bold(title),
            x=0.5,
            font=dict(family=FONT_FAMILY_BOLD, size=TITLE_SIZE),
        ),
        font=dict(family=FONT_FAMILY_BODY),
        margin=dict(l=100, r=70, t=140, b=100),
        legend=dict(
            font=dict(family=FONT_FAMILY_BODY, size=LEGEND_SIZE),
            title=dict(font=dict(family=FONT_FAMILY_BOLD, size=LEGEND_TITLE_SIZE)),
            itemsizing="constant",
            borderwidth=1,
        ),
    )

    fig.update_xaxes(
        title=dict(
            text=_bold(x_title) if x_title else None,
            font=dict(family=FONT_FAMILY_BOLD, size=AXIS_TITLE_SIZE),
        ),
        tickfont=dict(family=FONT_FAMILY_BOLD, size=TICK_SIZE),
        showline=True,
        linewidth=AXIS_LINE_WIDTH,
        mirror=True,
        showgrid=True,
        gridwidth=GRID_WIDTH,
        gridcolor="rgba(0,0,0,0.15)",
    )

    fig.update_yaxes(
        title=dict(
            text=_bold(y_title) if y_title else None,
            font=dict(family=FONT_FAMILY_BOLD, size=AXIS_TITLE_SIZE),
        ),
        tickfont=dict(family=FONT_FAMILY_BOLD, size=TICK_SIZE),
        showline=True,
        linewidth=AXIS_LINE_WIDTH,
        mirror=True,
        showgrid=True,
        gridwidth=GRID_WIDTH,
        gridcolor="rgba(0,0,0,0.15)",
    )

    return fig


def apply_colorbar_style(fig: go.Figure, title: str):
    """Uniform bold + large colorbar styling."""
    fig.update_layout(
        coloraxis_colorbar=dict(
            title=dict(
                text=_bold(title),
                font=dict(family=FONT_FAMILY_BOLD, size=COLORBAR_TITLE_SIZE),
            ),
            tickfont=dict(family=FONT_FAMILY_BOLD, size=COLORBAR_TICK_SIZE),
            thickness=COLORBAR_THICKNESS,
            len=0.85,
        )
    )
    return fig


# =========================================================
# A) DOMAIN SEPARABILITY (LINEAR PROBE)
# =========================================================
domains = df_all["domain"].astype(str).values
le = LabelEncoder()
y_dom = le.fit_transform(domains)
domain_names = list(le.classes_)

clf = LogisticRegression(
    penalty="l2",
    solver="lbfgs",
    max_iter=500,
    multi_class="auto",
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accs = []

for tr, te in cv.split(z, y_dom):
    clf.fit(z[tr], y_dom[tr])
    accs.append(clf.score(z[te], y_dom[te]))

accs = np.asarray(accs)
mean_acc = float(accs.mean())

sep_df = pd.DataFrame({
    "fold": np.arange(1, len(accs) + 1),
    "accuracy": accs
})

fig = px.bar(sep_df, x="fold", y="accuracy")

fig.update_traces(
    marker=dict(line=dict(width=BAR_EDGE_WIDTH, color="black")),
    text=sep_df["accuracy"].map(lambda x: f"{x:.2f}"),
    textposition="outside",
    textfont=dict(family=FONT_FAMILY_BOLD, size=18),
    cliponaxis=False,
)

fig.update_yaxes(range=[0, sep_df["accuracy"].max() * 1.15])

fig.add_hline(
    y=1.0 / len(domain_names),
    line_dash="dash",
    line_width=3,
    annotation_text=_bold("Random baseline"),
    annotation_position="bottom right",
    annotation_font=dict(
        family=FONT_FAMILY_BOLD,
        size=40,
        color="black",
    ),
)

fig = style_sup_2d(
    fig,
    title=f"Domain separability from latent space (mean = {mean_acc:.2f})",
    x_title="CV fold",
    y_title="Classification accuracy",
)

export_plotly(fig, PAPER_OUT, "FigA_S1_domain_separability_accuracy")


# =========================================================
# B) LATENT DIMENSION IMPORTANCE
# =========================================================
clf.fit(z, y_dom)
coef = clf.coef_
latent_importance = np.linalg.norm(coef, axis=0)

imp_df = pd.DataFrame({
    "latent_dim": [f"z{k}" for k in range(len(latent_importance))],
    "importance": latent_importance,
}).sort_values("importance", ascending=False)

imp_df.to_csv(PAPER_OUT / "PaperA_latent_dimension_importance.csv", index=False)

top_imp = imp_df.head(30)

fig = px.bar(top_imp, x="latent_dim", y="importance")

fig.update_traces(
    marker=dict(line=dict(width=BAR_EDGE_WIDTH, color="black")),
    text=top_imp["importance"].map(lambda x: f"{x:.2f}"),
    textposition="outside",
    textfont=dict(family=FONT_FAMILY_BOLD, size=16),
    cliponaxis=False,
)

fig.update_yaxes(range=[0, top_imp["importance"].max() * 1.15])

fig = style_sup_2d(
    fig,
    title="Latent dimensions ranked by domain importance",
    x_title="Latent dimension",
    y_title="‖Linear probe coefficient‖",
)

export_plotly(fig, PAPER_OUT, "FigA_S2_latent_dimension_importance")


# =========================================================
# C) MAP MOST DOMAIN-INFORMATIVE LATENT DIMS ONTO UMAP
# =========================================================
top_dims = top_imp["latent_dim"].iloc[:3]
top_idx = [int(d[1:]) for d in top_dims]

for k in top_idx:
    tmp = umap_df.copy()
    tmp["domain"] = df_all["domain"].astype(str).values
    tmp[f"z{k}"] = z[:, k]

    fig = px.scatter(
        tmp,
        x="UMAP1",
        y="UMAP2",
        color=f"z{k}",
        hover_data=["domain"],
        opacity=0.9,
    )

    fig.update_traces(
        marker=dict(size=MARKER_SIZE, line=dict(width=1.2, color="black"))
    )

    fig = style_sup_2d(
        fig,
        title=f"UMAP colored by latent dimension z{k}",
        x_title="UMAP1",
        y_title="UMAP2",
    )

    fig = apply_colorbar_style(fig, title=f"z{k}")

    export_plotly(fig, PAPER_OUT, f"FigA_S3_umap_colored_by_z{k}")


# =========================================================
# DONE
# =========================================================
print("\n✅ Supplementary analyses complete.")
print("📁 Outputs in:", PAPER_OUT.resolve())
print("🖼️ Export size:", EXPORT_WIDTH_PX, "×", EXPORT_HEIGHT_PX)


💾 Saved: FigA_S1_domain_separability_accuracy.png


💾 Saved: FigA_S2_latent_dimension_importance.png


💾 Saved: FigA_S3_umap_colored_by_z54.png


💾 Saved: FigA_S3_umap_colored_by_z55.png


💾 Saved: FigA_S3_umap_colored_by_z44.png

✅ Supplementary analyses complete.
📁 Outputs in: C:\Users\hrnbe\Greenbootcamps\Projects\Carbon&MOF\Paper 3\Paper A\models_disentangled\diagnostics\paperA_latent_transfer
🖼️ Export size: 1800 × 1350
